In [1]:
import json
import os
import pandas as pd

from common import *

exp_root = "/home/ict/appProfile/data/addtion_exp/standard_stress_cache_20231023031422/"

In [2]:
exp_json = os.path.join(exp_root, "exp.json")
exp_data = os.path.join(exp_root, "merged.csv")

In [3]:
df_total = pd.read_csv(exp_data, index_col=0)
df_total.set_index('Time', inplace=True)
df_total

,host_cache_llc_capacity_numa_0,host_cache_llc_capacity_numa_1,host_cache_llc_capacity_numa_2,host_cache_llc_capacity_numa_3,host_kernel_context_switch,host_kernel_interrupt,host_kernel_thread_fork_fork,host_kernel_syscall_accept,host_kernel_syscall_accept4,host_kernel_syscall_access,...,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,vm_block_io_sector_offset_offset_vda,vm_block_io_iops_iops_vda,app__qps_error,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
Time,,,,,,,,,,,,,,,,,,,,,
1698030862000,0.911932,0.991477,1.019886,0.960227,3821.500000,3472.000000,0.0,0.5,0.0,329.5,...,0,0.000000,700579840,0,0,0.0,0.0,0.0,0.000000,0.000000
1698030866000,0.977273,0.980114,0.997159,0.982955,126150.000000,109889.000000,3.5,0.5,2.5,421.5,...,0,0.000000,700579840,0,0,74036.0,7462.0,81498.0,0.000000,0.000000
1698030870000,0.963068,0.943182,0.985795,0.982955,140610.500000,118787.000000,0.0,0.5,0.0,329.5,...,0,0.000000,700579840,0,0,98575.0,9859.0,108434.0,2.691806,2.696858
1698030874000,0.974432,0.951705,0.991477,0.974432,135590.500000,116711.500000,0.0,0.5,0.0,329.5,...,1,0.000245,700579840,0,0,99164.0,9914.0,109078.0,2.556764,2.576941
1698030878000,0.957386,0.940341,0.991477,0.948864,93780.500000,87644.500000,0.0,0.5,0.0,329.5,...,0,0.000000,700579840,0,0,39840.0,3985.0,43825.0,2.452920,2.474957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1698040386000,0.883523,0.974432,0.985795,0.963068,50073.000000,45783.500000,0.0,0.5,0.0,329.5,...,0,0.000000,700710912,0,0,444737.5,44492.0,489229.5,33.670527,33.562198
1698040390000,0.926136,0.971591,0.980114,0.974432,16498.500000,16397.500000,0.0,0.5,0.0,329.5,...,0,0.000000,700710912,0,0,147978.0,14772.0,162750.0,33.865752,33.655758
1698040394000,0.985795,0.946023,0.994318,0.977273,4415.000000,5310.500000,0.0,0.5,0.0,329.5,...,0,0.000000,700710912,0,0,0.0,0.0,0.0,0.000000,0.000000


In [4]:
with open(exp_json, 'r') as f:
    exp = json.load(f)

In [5]:
exp["info_per_epoch"]
workload_info = exp["info_per_workload"]["redis_0"]["info_per_epoch"][0]

In [6]:
def format_to_13_timestamp(ts):
    assert ts + 1 < 10000000000000
    return int(format(ts, '0<13d'))
        

start = format_to_13_timestamp(workload_info["start_time"])
end = format_to_13_timestamp(workload_info["end_time"])

In [9]:
def filter_row_timerange(start, end):
    return lambda x : x.loc[(x.index >= start) & (x.index <=end)]

def filter_column_useless(std_min, mean_min=0):
    def __filter_column_useless(df):
        cols_to_drop = set()
        std = df.std()
        mean = df.mean()

        cols_to_drop.update(std[std <= std_min].index)
        cols_to_drop.update(mean[mean <= mean_min].index)
        return df.drop(columns = cols_to_drop)
    return __filter_column_useless

# same col may not change at all
# so it is important to set col prefix and choose those essential
def filter_column_noise(col_prefix, sparam=1.5, l=0.25, h=0.75):
    def __filter_column_noise(df):
        
        temp_df = filter_column_startswith(col_prefix)(df)
        q1 = temp_df.quantile(l)
        q3 = temp_df.quantile(h)
        iqr = q3 - q1
        outliers = ((temp_df < (temp_df - sparam * iqr)) | (temp_df > (q3 + sparam * iqr))).any(axis=1)
        return df.drop(temp_df[outliers].index)
    
    return __filter_column_noise

def filter_column_startswith(col_prefix):
    return lambda x : x[[col for col in x.columns if col.startswith(col_prefix)]]

def apply_filter(df, filters=[]):
    for filter in filters:
        df = filter(df)
    return df

In [10]:
df = apply_filter(df_total, filters=[
    filter_row_timerange(start, end),
    filter_column_startswith(col_prefix=("vm", "app")),
    filter_column_useless(std_min=1e-10),
    filter_column_noise(col_prefix=("app")),
])
df

,vm_hypervisor_emulator_syscall_count_clone3,vm_hypervisor_emulator_syscall_count_fdatasync,vm_hypervisor_emulator_syscall_count_futex_time64,vm_hypervisor_emulator_syscall_count_madvise,vm_hypervisor_emulator_syscall_count_munmap,vm_hypervisor_emulator_syscall_count_poll_time64,vm_hypervisor_emulator_syscall_count_ppoll_time64,vm_hypervisor_emulator_syscall_count_prctl,vm_hypervisor_emulator_syscall_count_pwrite,vm_hypervisor_emulator_syscall_count_pwritev,...,vm_block_io_bytes_io_write,vm_block_io_requests_write,vm_block_io_time_write,vm_block_io_flush_request_and_time_request_vda,vm_block_io_flush_request_and_time_time_vda,app_redis_qos_qps_of_redis_get,app_redis_qos_qps_of_redis_set,app_redis_qos_qps_of_redis_total,app_redis_qos_p99_latency_set,app_redis_qos_p99_latency_get
Time,,,,,,,,,,,,,,,,,,,,,
1698030882000,0.0,0.0,0.0,0.0,0.0,315.5,19.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,39967.0,3998.0,43965.0,0.990363,0.990399
1698030886000,0.0,0.0,0.0,0.0,0.0,315.5,18.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,40349.5,4032.5,44382.0,0.990121,0.990194
1698030890000,0.0,0.0,0.0,0.0,0.0,315.5,19.5,0.0,0.0,0.0,...,8192.0,2.0,0.000377,0,0.000000,40161.0,4012.0,44173.0,0.990879,0.990565
1698030894000,0.0,0.0,0.0,0.0,0.0,315.0,19.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,39884.5,3987.0,43871.5,0.991942,0.990764
1698030898000,0.0,0.0,4.5,2.0,1.5,315.5,18.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,39500.5,3949.5,43450.0,0.990742,0.990829
1698030902000,0.0,0.0,0.0,0.0,0.0,315.5,19.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,40177.5,4017.5,44195.0,0.990000,0.990000
1698030906000,0.0,0.0,0.0,0.0,0.0,315.5,19.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,39463.5,3947.0,43410.5,0.990252,0.990391
1698030910000,0.0,0.0,0.0,0.0,0.0,315.5,18.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,40448.5,4044.0,44492.5,0.990000,0.990098
1698030914000,0.0,0.0,0.0,0.0,0.0,315.5,19.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0,0.000000,40935.0,4092.0,45027.0,0.990000,0.990000
